# 生成网址

In [1]:
# 函数：生成网址  
# 返回值：url_list为网址列表
def generate_url():   # 添加：定义函数名
    url_list = []
    template = 'https://movie.douban.com/top250?start={num}'
    for p in range(1,11):
        url = template.format(num=(p-1)*25)
        url_list.append(url)
    return url_list        # 添加：返回网址列表

# 请求+获取网页数据	

In [2]:
import requests

# 函数：获得html  get_html(url)
# 参数说明：url为单个网址
# 返回值：html为网址的html数据，即网页源代码的字符串
def get_html(url):         # 添加：定义函数名
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
            AppleWebKit/537.36 (KHTML, like Gecko) \
           Chrome/119.0.0.0 Safari/537.36 Edg/119.0.0.0'}
    resp = requests.get(url,headers=headers)
    html = resp.text
    return html            # 添加：返回网址的html数据

# 解析数据

In [3]:
from pyquery import PyQuery as pq  #导入pyquery包
import re    # 导入re包

# 函数：解析数据  extract_movieinfo_list(html)
# 参数说明：html为网页源代码的字符串
# 返回值：bookinfo_list为书籍的字典列表
def extract_movieinfo_list(html):         # 添加：定义函数名
    movieinfo_list = []
    doc = pq(html)
    for movie in doc.items('div.info'):
        try:                             # 添加：try语句，避免特殊网页中断整个循环
            movie_name = movie('div.hd a').text()    
            desc = movie('span.inq').text()
            score = movie('span.rating_nums').text()
            
            # 提取导演信息
            director = doc('.bd p').text().split('导演: ')[1].split('&')[0].strip()
            # 提取演员信息
            actors = doc('.bd p').text().split('主演: ')[1].split('<br>')[0].strip().split(' / ')
            # 提取年份
            year = doc('.bd p').text().split('&nbsp;/&nbsp;')[0].strip()
            # 提取产出地
            place = doc('.bd p').text().split('&nbsp;/&nbsp;')[1].strip()

            # 提取电影类型
            genres = doc('.bd p').text().split('&nbsp;/&nbsp;')[2].strip().split(' ')
            
            people_num_raw = movie('span.pl').text()       
            people_num = re.findall('[0-9]+', people_num_raw)[0]

            movieinfo = {'movie_name':movie_name,
                         'director':director,
                         'actors':actors,
                         'year':year,
                         'place':place,
                         'score':score,
                         'people_num':people_num,
                          'desc':desc,
                        }

            movieinfo_list.append(movieinfo)
        except:                        # 添加：except和pass语句，如果碰到bug，那么跳出此次循环、不执行任何操作，进行下一次循环
            pass
        
    return movieinfo_list               # 添加：返回书籍的字典列表

# 存储数据

In [4]:
import csv     # 导入csv包

In [5]:
# 函数：爬虫主函数  main(filename)
# 参数说明：filename为文件名称
# 仅执行命令，不返回任何值
def main(filename):         # 添加：定义函数名
    print('开始采集豆瓣电影top 250！')              # 添加：说明“开始采集豆瓣读书top 250书籍！”
    
    # 生成所有网址url_list
    url_list = generate_url()
    
    # 打开文件
    file = open(filename, 'a+', encoding='utf-8', newline='')     # 修改：将文件名称，设定为函数的参数
    fieldnames = ['movie_name','director','actors','year','place','score','people_num','desc'] 
    writer = csv.DictWriter(file, fieldnames=fieldnames)    
    writer.writeheader() 

    # 对所有网址url_list循环步骤2-4
    for url in url_list:
        print('正在采集：{url}'.format(url=url))    # 添加：说明“正在采集<url>”
        html = get_html(url)
        movieinfo_list = extract_movieinfo_list(html)
        for movieinfo in movieinfo_list:
            writer.writerow(movieinfo)

    file.close()

    print('采集完毕！')            # 添加：说明“采集完毕！”

In [6]:
main(filename='movies_250.csv')

开始采集豆瓣电影top 250！
正在采集：https://movie.douban.com/top250?start=0
正在采集：https://movie.douban.com/top250?start=25
正在采集：https://movie.douban.com/top250?start=50
正在采集：https://movie.douban.com/top250?start=75
正在采集：https://movie.douban.com/top250?start=100
正在采集：https://movie.douban.com/top250?start=125
正在采集：https://movie.douban.com/top250?start=150
正在采集：https://movie.douban.com/top250?start=175
正在采集：https://movie.douban.com/top250?start=200
正在采集：https://movie.douban.com/top250?start=225
采集完毕！


# 参考：
- [Excel打开监控数据CSV文件乱码如何处理？_云监控服务 CES_常见问题_产品咨询_华为云 ](https://support.huaweicloud.com/ces_faq/ces_faq_0018.html)